In [1]:
import os
import sys
src_path = os.path.abspath('../')
if src_path not in sys.path:
    sys.path.append(src_path)

In [2]:
sys.path

['/home/mohammad/RenewableInsight/notebooks',
 '/home/mohammad/anaconda3/lib/python312.zip',
 '/home/mohammad/anaconda3/lib/python3.12',
 '/home/mohammad/anaconda3/lib/python3.12/lib-dynload',
 '',
 '/home/mohammad/anaconda3/lib/python3.12/site-packages',
 '/home/mohammad/RenewableInsight']

In [3]:
import pandas as pd
pd.options.mode.copy_on_write = True
from datetime import datetime, timedelta
import secrets
import string
from src.api.entsoe_api import ENTSOEAPI
from src.config import Config

In [4]:
data_downloader = ENTSOEAPI(
        year=int(Config.YEAR), 
        month=int(Config.MONTH), 
        country_code=Config.COUNTRY_CODE,
        api_key=Config.ENTSOE_API_KEY
    )

2024-08-03 11:43:31,495 - INFO - root - Logging setup complete using config: ./logs/logging.dev.ini


In [5]:
data_downloader.fetch_data(data_type='generation')
data = data_downloader.data

2024-08-03 11:43:33,489 - DEBUG - entsoe.entsoe - Performing request to https://web-api.tp.entsoe.eu/api with params {'documentType': 'A75', 'processType': 'A16', 'in_Domain': '10Y1001A1001A83F', 'securityToken': 'b307e891-a7a7-44a1-a7fa-4018009415c2', 'periodStart': '202406302200', 'periodEnd': '202407302200'}
2024-08-03 11:43:33,494 - DEBUG - urllib3.connectionpool - Starting new HTTPS connection (1): web-api.tp.entsoe.eu:443
2024-08-03 11:43:36,503 - DEBUG - urllib3.connectionpool - https://web-api.tp.entsoe.eu:443 "GET /api?documentType=A75&processType=A16&in_Domain=10Y1001A1001A83F&securityToken=b307e891-a7a7-44a1-a7fa-4018009415c2&periodStart=202406302200&periodEnd=202407302200 HTTP/11" 200 None
2024-08-03 11:43:46,961 - INFO - root - generation data has been successfully downloaded.


In [6]:
data.head(4)

,date,biomass_actual_aggregated,fossil_brown_coal_lignite_actual_aggregated,fossil_gas_actual_aggregated,fossil_hard_coal_actual_aggregated,fossil_oil_actual_aggregated,geothermal_actual_aggregated,hydro_pumped_storage_actual_aggregated,hydro_pumped_storage_actual_consumption,hydro_run-of-river_and_poundage_actual_aggregated,hydro_water_reservoir_actual_aggregated,other_actual_aggregated,other_renewable_actual_aggregated,solar_actual_aggregated,solar_actual_consumption,waste_actual_aggregated,wind_offshore_actual_aggregated,wind_onshore_actual_aggregated,wind_onshore_actual_consumption
0,2024-07-01 00:00:00+02:00,4127.0,9170.0,3390.0,1247.0,316.0,22.0,1175.0,26.0,1948.0,625.0,114.0,90.0,0.0,0.0,599.0,1207.0,4693.0,0.0
1,2024-07-01 00:15:00+02:00,4126.0,9114.0,3502.0,1268.0,316.0,22.0,792.0,36.0,1944.0,670.0,114.0,90.0,0.0,0.0,598.0,1358.0,4635.0,0.0
2,2024-07-01 00:30:00+02:00,4132.0,9123.0,3483.0,1313.0,316.0,22.0,481.0,353.0,1940.0,755.0,114.0,90.0,0.0,0.0,599.0,1406.0,4570.0,0.0
3,2024-07-01 00:45:00+02:00,4123.0,9086.0,3498.0,1321.0,316.0,22.0,271.0,391.0,1909.0,652.0,114.0,90.0,0.0,0.0,599.0,1439.0,4634.0,0.0


In [11]:
data.reset_index(drop=False, inplace=True)

In [13]:
data.rename(columns={'index':'date'}, inplace=True)

In [19]:
data.columns.to_list()


['date',
 'biomass_actual_aggregated',
 'fossil_brown_coal_lignite_actual_aggregated',
 'fossil_gas_actual_aggregated',
 'fossil_hard_coal_actual_aggregated',
 'fossil_oil_actual_aggregated',
 'geothermal_actual_aggregated',
 'hydro_pumped_storage_actual_aggregated',
 'hydro_pumped_storage_actual_consumption',
 'hydro_run-of-river_and_poundage_actual_aggregated',
 'hydro_water_reservoir_actual_aggregated',
 'other_actual_aggregated',
 'other_renewable_actual_aggregated',
 'solar_actual_aggregated',
 'solar_actual_consumption',
 'waste_actual_aggregated',
 'wind_offshore_actual_aggregated',
 'wind_onshore_actual_aggregated',
 'wind_onshore_actual_consumption']

In [14]:
data.columns = ['_'.join(filter(None, col)).lower().replace(' ', '_').replace('/', '_') for col in data.columns]

In [15]:
data.columns

Index(['d_a_t_e', 'b_i_o_m_a_s_s___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',
       'f_o_s_s_i_l___b_r_o_w_n___c_o_a_l___l_i_g_n_i_t_e___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',
       'f_o_s_s_i_l___g_a_s___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',
       'f_o_s_s_i_l___h_a_r_d___c_o_a_l___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',
       'f_o_s_s_i_l___o_i_l___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',
       'g_e_o_t_h_e_r_m_a_l___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',
       'h_y_d_r_o___p_u_m_p_e_d___s_t_o_r_a_g_e___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',
       'h_y_d_r_o___p_u_m_p_e_d___s_t_o_r_a_g_e___a_c_t_u_a_l___c_o_n_s_u_m_p_t_i_o_n',
       'h_y_d_r_o___r_u_n_-_o_f_-_r_i_v_e_r___a_n_d___p_o_u_n_d_a_g_e___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',
       'h_y_d_r_o___w_a_t_e_r___r_e_s_e_r_v_o_i_r___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',
       'o_t_h_e_r___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',
       'o_t_h_e_r___r_e_n_e_w_a_b_l_e___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',
       's_o_l_a_r___a_c_t_u_a_l___a_g_g_r_e_g_a_t_e_d',

In [2]:
df = pd.read_csv("/home/mohammadp/RenewableInsight/data/2024_03_ActualTotalLoad_6.1.A.csv",delimiter="\t")

In [3]:
df.columns

Index(['DateTime', 'ResolutionCode', 'AreaCode', 'AreaTypeCode', 'AreaName',
       'MapCode', 'TotalLoadValue', 'UpdateTime'],
      dtype='object')

In [4]:
df.head(5)

,DateTime,ResolutionCode,AreaCode,AreaTypeCode,AreaName,MapCode,TotalLoadValue,UpdateTime
0,2024-03-10 22:00:00.000,PT15M,10YDE-ENBW-----N,CTA,DE(TransnetBW) CTA,DE_TransnetBW,6133.87,2024-03-10 23:31:05.005
1,2024-03-10 21:30:00.000,PT15M,10YHR-HEP------M,CTA,HR CTA,HR,1968.00,2024-03-11 10:46:36.036
2,2024-03-10 21:30:00.000,PT15M,10YHR-HEP------M,BZN,HR BZN,HR,1968.00,2024-03-11 11:01:53.053
3,2024-03-10 22:00:00.000,PT15M,10YRO-TEL------P,BZN,RO BZN,RO,5383.00,2024-03-10 23:31:08.008
4,2024-03-10 21:00:00.000,PT60M,10YCS-SERBIATSOV,CTA,RS CTA,RS,4086.00,2024-03-11 08:02:18.018


In [5]:
[mapcode for mapcode in df.MapCode.unique() if mapcode[:2]=='DE']

['DE_TransnetBW', 'DE_Amprion', 'DE', 'DE_50HzT', 'DE_TenneT_GER', 'DE_LU']

In [7]:
df_DE = df[df.MapCode.isin([mapcode for mapcode in df.MapCode.unique() if mapcode[:2]=='DE'])]

In [8]:
df_DE.head(5)

,DateTime,ResolutionCode,AreaCode,AreaTypeCode,AreaName,MapCode,TotalLoadValue,UpdateTime
0,2024-03-10 22:00:00.000,PT15M,10YDE-ENBW-----N,CTA,DE(TransnetBW) CTA,DE_TransnetBW,6133.87,2024-03-10 23:31:05.005
12,2024-03-10 22:15:00.000,PT15M,10YDE-ENBW-----N,CTA,DE(TransnetBW) CTA,DE_TransnetBW,6003.32,2024-03-10 23:46:13.013
17,2024-03-10 22:15:00.000,PT15M,10YDE-RWENET---I,CTA,DE(Amprion) CTA,DE_Amprion,16640.27,2024-03-11 18:17:01.001
23,2024-03-10 22:00:00.000,PT15M,10Y1001A1001A83F,CTY,DE CTY,DE,46343.59,2024-03-11 18:31:54.054
24,2024-03-10 22:30:00.000,PT15M,10YDE-ENBW-----N,CTA,DE(TransnetBW) CTA,DE_TransnetBW,5857.35,2024-03-11 00:01:47.047


In [13]:
df_DE.AreaName.unique()

array(['DE(TransnetBW) CTA', 'DE(Amprion) CTA', 'DE CTY',
       'DE(50Hertz) CTA', 'DE(TenneT DE) CTA', 'DE-LU BZN'], dtype=object)

In [7]:
df_DE['DateTime'] = pd.to_datetime(df_DE.DateTime,format='%Y-%m-%d %H:%M:%S.%f')

In [8]:
df_DE.loc[:,'DateTime'] = pd.to_datetime(df_DE.DateTime,format='%Y-%m-%d %H:%M:%S.%f')

In [9]:
df_DE.loc[:,'Day'] = df_DE.DateTime.dt.day
df_DE.loc[:,'Month'] = df_DE.DateTime.dt.month
df_DE.loc[:,'Year'] = df_DE.DateTime.dt.year
df_DE.loc[:,'Hour'] = df_DE.DateTime.dt.hour
df_DE.loc[:,'Minute'] = df_DE.DateTime.dt.minute

In [10]:
df_DE.Day.unique().shape

(27,)

In [11]:
df_DE.Month.unique()

array([3], dtype=int32)

In [12]:
df_DE.head(5)

,DateTime,ResolutionCode,AreaCode,AreaTypeCode,AreaName,MapCode,TotalLoadValue,UpdateTime,Day,Month,Year,Hour,Minute
0,2024-03-10 22:00:00,PT15M,10YDE-ENBW-----N,CTA,DE(TransnetBW) CTA,DE_TransnetBW,6133.87,2024-03-10 23:31:05.005,10,3,2024,22,0
12,2024-03-10 22:15:00,PT15M,10YDE-ENBW-----N,CTA,DE(TransnetBW) CTA,DE_TransnetBW,6003.32,2024-03-10 23:46:13.013,10,3,2024,22,15
17,2024-03-10 22:15:00,PT15M,10YDE-RWENET---I,CTA,DE(Amprion) CTA,DE_Amprion,16640.27,2024-03-11 18:17:01.001,10,3,2024,22,15
23,2024-03-10 22:00:00,PT15M,10Y1001A1001A83F,CTY,DE CTY,DE,46343.59,2024-03-11 18:31:54.054,10,3,2024,22,0
24,2024-03-10 22:30:00,PT15M,10YDE-ENBW-----N,CTA,DE(TransnetBW) CTA,DE_TransnetBW,5857.35,2024-03-11 00:01:47.047,10,3,2024,22,30


In [15]:
df_DE = df_DE.groupby(by=['Day','Month','Year','Hour','Minute'])[['TotalLoadValue']].sum().reset_index()

In [21]:
df_DE.dtypes

Day                  int8
Month                int8
Year                int16
Hour                 int8
Minute               int8
TotalLoadValue    float64
dtype: object

In [20]:
df_DE.Month = df_DE.Month.astype('int8')
df_DE.Year = df_DE.Year.astype('int16')
df_DE.Hour = df_DE.Hour.astype('int8')
df_DE.Minute = df_DE.Minute.astype('int8')
df_DE.Day = df_DE.Day.astype('int8')

In [13]:
today = datetime.now()
last_day = today - timedelta(days=5)
dates = [last_day - timedelta(days=i) for i in range(5)]

In [35]:
def create_s3_keys_dates(data_item_name,data_item_no):
    """
    Generate S3 object keys with embedded dates and a random string, formatted specifically for use as filenames.
    
    Parameters:
        data_item_name (str): Base name of the data item to include in the key.
        data_item_no (int): Data item number to include in the key.
        
    Yields:
        tuple: A tuple containing the S3 object key and the corresponding date object for each key.
    
    The function calculates dates from 5 days ago and generates five S3 keys, one for each day starting from
    'last_day' (5 days ago) to 'last_day-4' (9 days ago). Each key includes the `data_item_name`, `data_item_no`,
    the date (day, month, year), and a random string suffix, stored in a .parquet file format in respective date folders.
    """
    today = datetime.now()
    last_day = today - timedelta(days=5)
    for i in range(5):
        date = last_day - timedelta(days=i)
        random_part = generate_random_string(10)
        object_key = f"{data_item_name}_{data_item_no}_{date.day}_{date.month}_{date.year}/{random_part}.parquet"
        yield object_key,date

In [37]:
def generate_random_string(n=10):
    """
    Generate a random string of lowercase letters and digits.
    
    Parameters:
        n (int): Length of the random string to generate. Default is 10.
        
    Returns:
        str: Random string of specified length.
    """
    return ''.join(secrets.choice(string.ascii_lowercase + string.digits) for _ in range(n))

In [40]:
df = df_DE
for object_key,date in create_s3_keys_dates('data_item_name','data_item_no'):
        df_date = df[(df.Day==date.day)&(df.Month==date.month)&(df.Year==date.year)]
        print(object_key)

data_item_name_data_item_no_7_4_2024/gmxtepz2kt.parquet
data_item_name_data_item_no_6_4_2024/bmj4qwvdcx.parquet
data_item_name_data_item_no_5_4_2024/4qxfqtfzyj.parquet
data_item_name_data_item_no_4_4_2024/i51ki8n5qh.parquet
data_item_name_data_item_no_3_4_2024/d8f1sr3ylm.parquet


In [14]:
df = pd.read_csv("/home/mohammadp/RenewableInsight/data/2024_03_ActualGenerationOutputPerGenerationUnit_16.1.A.csv",delimiter="\t")

In [15]:
df.columns

Index(['DateTime', 'ResolutionCode', 'AreaCode', 'AreaTypeCode', 'AreaName',
       'MapCode', 'GenerationUnitEIC', 'PowerSystemResourceName',
       'ProductionType', 'ActualGenerationOutput', 'ActualConsumption',
       'InstalledGenCapacity', 'UpdateTime'],
      dtype='object')

In [16]:
df.head(5)

,DateTime,ResolutionCode,AreaCode,AreaTypeCode,AreaName,MapCode,GenerationUnitEIC,PowerSystemResourceName,ProductionType,ActualGenerationOutput,ActualConsumption,InstalledGenCapacity,UpdateTime
0,2024-03-01 00:00:00.000,PT60M,10YES-REE------0,CTA,ES CTA,ES,18W000000000K82B,FV1014,Solar,0.0,NaN,157.5,2024-03-04 18:35:05.005
1,2024-03-01 01:00:00.000,PT60M,10YES-REE------0,CTA,ES CTA,ES,18W000000000K82B,FV1014,Solar,0.0,NaN,157.5,2024-03-04 18:35:05.005
2,2024-03-01 02:00:00.000,PT60M,10YES-REE------0,CTA,ES CTA,ES,18W000000000K82B,FV1014,Solar,0.0,NaN,157.5,2024-03-04 18:35:05.005
3,2024-03-01 03:00:00.000,PT60M,10YES-REE------0,CTA,ES CTA,ES,18W000000000K82B,FV1014,Solar,0.0,NaN,157.5,2024-03-04 18:35:05.005
4,2024-03-01 04:00:00.000,PT60M,10YES-REE------0,CTA,ES CTA,ES,18W000000000K82B,FV1014,Solar,0.0,NaN,157.5,2024-03-04 18:35:05.005


In [19]:
df_DE = df[df.MapCode.isin([mapcode for mapcode in df.MapCode.unique() if mapcode[:2]=='DE'])]

In [23]:
df_DE.PowerSystemResourceName.unique()

array(['HAMBORN-4', 'Block F', 'Block GT2', 'DT HKW-Niehl-2', 'HKW IIIB',
       'KMW_KW2DT', 'KW Hamm-Uentrop Block 10', 'KW5', 'Niederaußem G',
       'Radag', 'VOELKLINGEN_HKV', 'Säckingen', 'VOELKLINGEN_MKV',
       'Vianden 3', 'Vianden 9', 'GuD-Anlage-HKW-Merkenich',
       'BERGKAMEN_A', 'Block AGuD', 'Block GT 12', 'HERNE_4',
       'Huckingen B', 'KMW_KW3', 'KMW_KW2GT', 'KW Hamm-Uentrop Block 20',
       'KW Lünen Block 1', 'Knapsack 1', 'Neurath E', 'Niederaußem E',
       'Niederaußem F', 'RUHRORT-4', 'Niederaußem K (BoA 1)', 'Vianden 1',
       'Vianden 2', 'Vianden 7', 'Vianden 8', 'WALSUM_10', 'Block GT 11',
       'Block GT1', 'HAMBORN-5', 'Emsland B', 'Huckingen A', 'Emsland D',
       'Koepchenwerk', 'Neurath C', 'Neurath D', 'Scholven 1',
       'Scholven B', 'Scholven C', 'Vianden 11', 'Vianden 4',
       'Weisweiler G', 'Weisweiler VGT - BI. H', 'KW Boxberg Block Q',
       'KW Lippendorf Block S', 'KW Boxberg Block N',
       'KW Boxberg Block P', 'KW Jänschwalde B

In [24]:
day = 4
month = 10

In [25]:
print(f'{day}')

4


In [28]:
day = 30
month = 4

formatted_day = f"{day:02}"
formatted_month = f"{month:02}"

print(f"Day: {formatted_day}")
print(f"Month: {formatted_month}")

Day: 30
Month: 04
